In [2]:
!pip install networkx powerlaw

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import statistics
import sys
import random
import math
import numpy as np
import powerlaw
import pandas as pd
import seaborn as sns
from collections import Counter
from scipy.stats import norm
from termcolor import colored, cprint
from matplotlib.patches import Rectangle

In [4]:
MORENO_RESIDENCE_HALL_FILE = 'networks/moreno_residence_hall.txt'

# Residence-Hall-Network

(Directed)

His directed network contains friendship ratings between 217 residents living at a residence hall located on the Australian National University campus. A node represents a person and 
an edge represents a friendship tie.  The friendships are weighted as follows from strongest to weakest tie:  5 (best friend), 4 (close friend), 3 (friend), 2, 1.

In [5]:
G = nx.read_weighted_edgelist(MORENO_RESIDENCE_HALL_FILE, create_using= nx.DiGraph, nodetype=int)
cprint(nx.info(G),'green')

Name: 
Type: DiGraph
Number of nodes: 217
Number of edges: 2672
Average in degree:  12.3134
Average out degree:  12.3134


In [6]:
pos = nx.spring_layout(G, k=0.6, iterations=100)
plt.figure(3, figsize=(100, 100))
edges, weights = zip(*nx.get_edge_attributes(G, 'weight').items())
nx.draw(G,node_color='#add8e6', node_size=8000,arrowsize=40, pos=pos, edgelist = edges, edge_color = weights, edge_cmap = plt.cm.Greys, edge_vmin=0.0, edge_vmax=5.0, width=2.0)
nx.draw_networkx_labels(G, font_size=30, pos=pos)
ax=plt.gca()
ax.collections[0].set_edgecolor("#000000")
plt.show()


KeyboardInterrupt: 

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
nodes = G.nodes()
edges = G.edges()

# Componenti connesse

La rete non è nè una componente fortemente connessa nè una componente debolmente connessa.

In [ ]:
cprint("The network is a unique stongly connected component: ",'blue', end=' ') 
cprint(str(nx.is_strongly_connected(G)), 'red')
cprint("The network is a unique weakly connected component: ", 'blue', end=' ')
cprint(str(nx.is_weakly_connected(G)), 'green')

In [ ]:
weakly_components = list(nx.weakly_connected_components(G))
cprint("Number of components: ", 'blue', end=' ') 
cprint(str(len(weakly_components)),'green')

largest_weakly = max(weakly_components, key=len)
cprint("Largest component: ", 'blue', end=' ') 
cprint(str(len(largest_weakly)) + " nodes", 'green')

## Componenti fortemente connesse

In [ ]:
strongly_components = list(nx.strongly_connected_components(G))
largest_strongly = max(strongly_components, key=len)

color_map = ['green' if node in largest_strongly else 'red' for node in G] 
edges, weights = zip(*nx.get_edge_attributes(G, 'weight').items())

plt.figure(4, figsize=(100, 100))
edges, weights = zip(*nx.get_edge_attributes(G, 'weight').items())
nx.draw(G,node_color=color_map, node_size=8000,arrowsize=40, pos=pos, edgelist = edges, edge_color = weights, edge_cmap = plt.cm.Greys, edge_vmin=0.0, edge_vmax=5.0, width=2.0)
nx.draw_networkx_labels(G, font_size=30, pos=pos)
ax=plt.gca()
ax.collections[0].set_edgecolor("#000000")


colors = ["g", "r"]
texts = ["Strongly connected nodes", "Weakly connected nodes"]
patches = [ plt.plot([],[], marker="o", ms=80, ls="", mec=None, color=colors[i], 
            label="{:s}".format(texts[i]) )[0]  for i in range(len(texts)) ]
patches.append(Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0, label=('Size of largest strongest connected component: '+ str(len(largest_strongly)))))
patches.append(Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0, label=('Number of components: '+ str(len(strongly_components)))))
plt.legend(handles=patches,loc=2, ncol=1, facecolor="#add8e6", numpoints=1,fontsize=70)

plt.show()
plt.show()

# Degree

In [ ]:
degrees_sequences = {'Degree sequence':[G.degree(n) for n in G.nodes],
'Strength degree sequence':[G.degree(n, weight='weight') for n in G.nodes],
'In degree sequence':[G.in_degree(n) for n in G.nodes],
'Out degree sequence':[G.out_degree(n) for n in G.nodes],
'In-Strength sequence':[G.in_degree(n, weight='weight') for n in G.nodes],
'Out-Strength degree sequence':[G.out_degree(n, weight='weight') for n in G.nodes]}

In [ ]:
def print_degree_statistics(mu_degree, median_degree, variance_degree,sigma_degree):
    cprint('Mean degree:', 'blue', end=' ') 
    cprint(mu_degree, 'green')
    cprint('Median degree:','blue', end=' ') 
    cprint(median_degree, 'green')
    cprint('Max degree:', 'blue', end=' ')
    cprint(max(degrees_sequences['Degree sequence']), 'green')
    cprint('Variance:', 'blue', end=' ')
    cprint(variance_degree, 'green')
    cprint('Standard deviation:', 'blue', end=' ')
    cprint(sigma_degree, 'green')
    print()

    highest_degree_node = max(G.nodes, key=G.degree)
    cprint("Highest degree node label:",'blue', end=' ')
    cprint(highest_degree_node,'green')
    cprint("Maximum degree", 'blue', end=' ')
    cprint(max(degrees_sequences['Degree sequence']),'green')


## Network statistics without weight

In [ ]:
mu_degree = statistics.mean(degrees_sequences['Degree sequence'])
median_degree = statistics.median(degrees_sequences['Degree sequence'])
variance_degree = statistics.variance(degrees_sequences['Degree sequence'])
sigma_degree = statistics.stdev(degrees_sequences['Degree sequence'])

print_degree_statistics(mu_degree, median_degree, variance_degree,sigma_degree)

## Network statistics with weight

In [ ]:
mu_degree = statistics.mean(degrees_sequences['Strength degree sequence'])
median_degree = statistics.median(degrees_sequences['Strength degree sequence'])
variance_degree = statistics.variance(degrees_sequences['Strength degree sequence'])
sigma_degree = statistics.stdev(degrees_sequences['Strength degree sequence'])

print_degree_statistics(mu_degree, median_degree, variance_degree,sigma_degree)

## Degree distribution

### Frequency

In [ ]:
degree_map = {}
for k in degrees_sequences.keys():
    degree_counts = Counter(degrees_sequences[k])
    plot_x = list(np.arange(min(degree_counts.keys()), max(degree_counts.keys())+1))
    plot_y = [degree_counts.get(x, 0) for x in plot_x]
    degree_map[k] = { 'x': plot_x, 'y': plot_y}

titles = list(degrees_sequences.keys())

fig_degree, axes = plt.subplots(nrows=3, ncols=2)
for i, ax in enumerate(axes.flatten()):
    #ax.bar(degree_map[titles[i]][0],degree_map[titles[i]][1])
    sns.histplot(ax=ax, data=degrees_sequences[titles[i]], kde=True, stat='frequency', bins=degree_map[titles[i]]['x'])
    ax.set_title(titles[i],fontdict= { 'fontsize': 16, 'fontweight':'bold'})
    ax.set_xlabel("Degree k")
    ax.set_ylabel("Frequency")

fig_degree.figure.set_size_inches((30,20))

### Probability

In [ ]:
def get_degree_probability(degree_counts, min_degree, max_degree):
  degree_prob = {}

  for degree in np.arange(min_degree, max_degree + 1):
    if degree in degree_counts:
      num_nodes_degree = degree_counts[degree]
      prob = num_nodes_degree/num_nodes
      degree_prob[degree] = prob
    else:
      degree_prob[degree] = 0
  
  return degree_prob



fig_prob, axes = plt.subplots(nrows=3, ncols=2)
for i, ax in enumerate(axes.flatten()):
    #ax.bar(degree_map[titles[i]][0],degree_map[titles[i]][1])
    degree_counts = Counter(degrees_sequences[titles[i]])
    degree_probability = get_degree_probability(degree_counts,min(degree_counts.keys()), max(degree_counts.keys()))
    sns.lineplot(ax=ax, data=degree_probability)
    ax.set_title(titles[i],fontdict= { 'fontsize': 16, 'fontweight':'bold'})
    ax.set_xlabel("Degree k")
    ax.set_ylabel("Probability")

fig_prob.figure.set_size_inches((30,20))  


# Power Law and Scale Free Regime

In [ ]:
# value of cdf between one, two
# and three S.D. around the mean
one_sd = norm.cdf(sigma_degree, mu_degree, sigma_degree) - norm.cdf(-sigma_degree, mu_degree, sigma_degree)
two_sd = norm.cdf(2 * sigma_degree, mu_degree, sigma_degree) - norm.cdf(-2 * sigma_degree, mu_degree, sigma_degree)
three_sd = norm.cdf(3 * sigma_degree, mu_degree, sigma_degree) - norm.cdf(-3 * sigma_degree, mu_degree, sigma_degree)
 
# printing the value of fractions
# within each band
cprint("Fraction of values within one SD:", 'blue', end=' ')
cprint(round(one_sd,2),'green')
cprint("Fraction of values within two SD:", 'blue', end=' ')
cprint(round(two_sd,2), 'green')
cprint("Fraction of values within three SD:", 'blue', end=' ') 
cprint(round(three_sd,2), "green")

if(one_sd==0.68 and two_sd==0.95 and three_sd==0.99):
    cprint('Bell shape distribution.', "green")
else:
    cprint('Doesn\'t fit Bell shape distribution.', "yellow")

In [ ]:
fit = powerlaw.Fit(degrees_sequences['Degree sequence'], xmin=28) 

print(fit.power_law.alpha)
print(fit.power_law.xmin)


fig2 = fit.plot_pdf(color='b', linewidth=2)
fig3 = fit.power_law.plot_pdf(color='g', linestyle='--', ax=fig2)
fig3.legend(["Probability density function","power law fit"])

R, p = fit.distribution_compare('power_law', 'exponential', normalized_ratio=True)
print (R, p)

plt.figure(figsize=(10, 6))
fit.distribution_compare('power_law', 'lognormal')
fig4 = fit.plot_ccdf(linewidth=3, color='black')
fit.power_law.plot_ccdf(ax=fig4, color='r', linestyle='--') #powerlaw
fit.lognormal.plot_ccdf(ax=fig4, color='g', linestyle='--') #lognormal
fit.stretched_exponential.plot_ccdf(ax=fig4, color='b', linestyle='--') #stretched_exponential
fig4.legend(["Probability density function","power law fit", "lognormal", "stretched exponential"])

# Centrality Measures

## Closeness Centrality 

In a connected graph, closeness centrality (or closeness) of a node is a measure of centrality in a network, calculated as the reciprocal of the sum of the length of the shortest paths between the node and all other nodes in the graph. Thus, the more central a node is, the closer it is to all other nodes.
The closeness centrality is normalized to (n-1)/(|G|-1) where n is the number of nodes in the connected part of graph containing the node. If the graph is not completely connected, this algorithm computes the closeness centrality for each connected part separately scaled by that parts size.
If the ‘distance’ keyword is set to an edge attribute key then the shortest-path length will be computed using Dijkstra’s algorithm with that edge attribute as the edge weight.

In [ ]:
def print_closeness(closeness):
    closeness_sequence = list(closeness.values())
    # closeness_sequence
    highest_closeness_node = max(G.nodes, key=closeness.get) #node id with highest closeness
    value_highest_closeness_node = closeness[highest_closeness_node] #value of highest closeness
    cprint('Node with highest closeness:', 'blue', end=' ')
    cprint(highest_closeness_node,'green')
    cprint('Value of highest closeness:', 'blue', end=' ')
    cprint(value_highest_closeness_node,'green')
    cprint('Mean closeness:', 'blue', end=' ')
    cprint(statistics.mean(closeness_sequence),'green')
    cprint('Median closeness:', 'blue', end=' ')
    cprint(statistics.median(closeness_sequence),'green')

### Incoming Closeness Centrality

In [ ]:
in_closeness = nx.centrality.closeness_centrality(G)
print_closeness(in_closeness)

### Outcoming Closeness Centrality

In [ ]:
out_closeness = nx.centrality.closeness_centrality(G.reverse())
print_closeness(out_closeness)

## Closeness Distribution

In [ ]:
fig_closs, axes = plt.subplots(nrows=1, ncols=2)

#plotting first graph
sns.histplot(data=list(in_closeness.values()), bins=50, ax=axes[0]) #with edge weight
axes[0].set_title("Incoming Closeness",fontdict= { 'fontsize': 16, 'fontweight':'bold'})
axes[0].set_xlabel("Closeness")
axes[0].set_ylabel("Number of nodes")

#plotting second graph
sns.histplot(data=list(out_closeness.values()), bins=50, ax=axes[1]) #with edge weight
axes[1].set_title("Outcoming Closeness",fontdict= { 'fontsize': 16, 'fontweight':'bold'})
axes[1].set_xlabel("Closeness")
axes[1].set_ylabel("Number of nodes")

fig_closs.figure.set_size_inches((15,5))

## Betweenness of nodes

In graph theory, betweenness centrality (or "betweeness centrality") is a measure of centrality in a graph based on shortest paths. For every pair of vertices in a connected graph, there exists at least one shortest path between the vertices such that either the number of edges that the path passes through (for unweighted graphs) or the sum of the weights of the edges (for weighted graphs) is minimized. The betweenness centrality for each vertex is the number of these shortest paths that pass through the vertex.
We ignore the weight in our case because they aren't meaningful.

In [ ]:
betweenness_nodes = nx.betweenness_centrality(G) #betweenness of all nodes in the network
highest_betweenness_node = max(G.nodes, key=betweenness_nodes.get) #node id with highest betweenness
value_highest_betweenness_node = betweenness_nodes[highest_betweenness_node] #value of highest node betweenness
betweenness_nodes_sequence = list(betweenness_nodes.values())
cprint('Node with highest betweenness:', 'blue', end=' ') 
cprint(highest_betweenness_node, 'green')
cprint('Value of highest betweenness node:', 'blue', end=' ') 
cprint(value_highest_betweenness_node, 'green')
cprint('Mean betweenness of nodes:', 'blue', end=' ') 
cprint(statistics.mean(betweenness_nodes_sequence), 'green')
cprint('Median betweenness of nodes:', 'blue', end=' ') 
cprint(statistics.median(betweenness_nodes_sequence), 'green')

## Betweenness Of Nodes Distribution 

In [ ]:
sns.histplot(data=betweenness_nodes_sequence, bins=50)
plt.ylabel("Number of nodes")
plt.xlabel("Node Betweenness")
plt.show()

## Betweenness of edges

Link betweenness: fraction of shortest paths among all possible node pairs that pass 
through the link.
Weights are used to calculate weighted shortest paths, so they are interpreted as distances.

In [ ]:
betweenness_edges = nx.centrality.edge_betweenness(G)
highest_betweenness_edge = max(G.edges, key=betweenness_edges.get) #edge id with highest betweenness
value_highest_betweenness_edge = betweenness_edges[highest_betweenness_edge] #value of highest edg betweenness
betweenness_edges_sequence = list(betweenness_edges.values())
cprint('Edge with highest betweenness:', 'blue', end=' ') 
cprint(highest_betweenness_edge, 'green')
cprint('Value of highest betweenness edge:', 'blue', end=' ') 
cprint(value_highest_betweenness_edge, 'green')
cprint('Mean betweenness of edges:', 'blue', end=' ') 
cprint(statistics.mean(betweenness_edges_sequence), 'green')
cprint('Median betweenness of edges:', 'blue', end=' ') 
cprint(statistics.median(betweenness_edges_sequence), 'green')

## Betweenness Of Edges Distribution

In [ ]:
sns.histplot(data=betweenness_edges_sequence, bins=50)
plt.ylabel("Number of edges")
plt.xlabel("Edge Betweenness")
plt.show()

## Assortativity: Correlation

Assortativity is the tendency in networks where nodes connect with other nodes similar to themselves. 
<br>Degree assortativity can be quantified as a Pearson correlation.

*   r = 1:  assortative
*   r = 0:  neutral (non-assortative)
*   r = -1: disassortative

## Assortativity: Weighted Degree Correlation

In [ ]:
matrix_k_k2=[]

for n in G.nodes:
    matrix_k_k2.append([G.degree(n,weight='weight'),nx.average_neighbor_degree(G, nodes=G.subgraph(n), weight='weight')[n]])
   
g = sns.regplot(data=pd.DataFrame(np.array(matrix_k_k2), columns=['weighted k', 'knn(k)']), x="weighted k", y="knn(k)",ci=None)
g.set_title("Strength Assortativity",fontdict= { 'fontsize': 16, 'fontweight':'bold'})
#g.set_yscale("log")
#g.set_xscale("log")


## Degree Correlation

In [ ]:
matrix_k_k2=[]

for n in G.nodes:
    matrix_k_k2.append([G.degree(n),nx.average_neighbor_degree(G, nodes=G.subgraph(n))[n]])

g = sns.regplot(data=pd.DataFrame(np.array(matrix_k_k2), columns=['k', 'knn(k)']), x="k", y="knn(k)",ci=None)
g.set_title("Assortativity",fontdict= { 'fontsize': 16, 'fontweight':'bold'})
#g.set_yscale("log")
#g.set_xscale("log")


## Assortativity Correlation Coefficient

In [ ]:
assortativity_data_1 = pd.DataFrame({'Type':'out-in', 'Pearson correlation coefficient' : [nx.degree_pearson_correlation_coefficient(G, x='out', y='in')]})
assortativity_data_2 = pd.DataFrame({'Type':'in-out', 'Pearson correlation coefficient' : [nx.degree_pearson_correlation_coefficient(G, x='in', y='out')]})
assortativity_data_3 = pd.DataFrame({'Type':'out-out', 'Pearson correlation coefficient' : [nx.degree_pearson_correlation_coefficient(G, x='out', y='out')]})
assortativity_data_4 = pd.DataFrame({'Type':'in-in', 'Pearson correlation coefficient' : [nx.degree_pearson_correlation_coefficient(G, x='in', y='in')]})

assortativity_data = pd.concat([assortativity_data_1,assortativity_data_2,assortativity_data_3,assortativity_data_4])

w_assortativity_data_1 = pd.DataFrame({'Type':'out-in', 'Pearson correlation coefficient' : [nx.degree_pearson_correlation_coefficient(G, x='out', y='in', weight='weight')]})
w_assortativity_data_2 = pd.DataFrame({'Type':'in-out', 'Pearson correlation coefficient' : [nx.degree_pearson_correlation_coefficient(G, x='in', y='out', weight='weight')]})
w_assortativity_data_3 = pd.DataFrame({'Type':'out-out', 'Pearson correlation coefficient' : [nx.degree_pearson_correlation_coefficient(G, x='out', y='out', weight='weight')]})
w_assortativity_data_4 = pd.DataFrame({'Type':'in-in', 'Pearson correlation coefficient' : [nx.degree_pearson_correlation_coefficient(G, x='in', y='in', weight='weight')]})

w_assortativity_data = pd.concat([w_assortativity_data_1,w_assortativity_data_2,w_assortativity_data_3,w_assortativity_data_4])

#Creating figure area
fig_ass, axes = plt.subplots(nrows=1, ncols=2)

#plotting first graph
sns.barplot(
    data=assortativity_data,
    x='Type', y='Pearson correlation coefficient',
    ax=axes[0]
)
axes[0].set_title('Unweighted',fontdict= { 'fontsize': 16, 'fontweight':'bold'})

#plotting second graph
sns.barplot(
    data=w_assortativity_data,
    x='Type', y='Pearson correlation coefficient',
    ax=axes[1]
)
axes[1].set_title('Weighted',fontdict= { 'fontsize': 16, 'fontweight':'bold'})


fig_ass.figure.set_size_inches((15,5))


In [ ]:
asso_test=nx.degree_pearson_correlation_coefficient(G)
if(round(asso_test,1)>=0.1): cprint('Assortative network', 'yellow')
if(round(asso_test,1)==0): cprint('Neutral network', 'yellow')
if(round(asso_test,1)<0): cprint('Disassortative network', 'yellow')


## k-nearest neighbours 
The average degree connectivity is the average nearest neighbor degree of nodes with degree k.

In [ ]:
avg_weighted_degree_connectivity = nx.average_degree_connectivity(G,weight='weight')
avg_degree_connectivity = nx.average_degree_connectivity(G)



#Creating figure area
fig_knearest, axes = plt.subplots(nrows=1, ncols=2)

sns.scatterplot(data=avg_degree_connectivity,
    ax=axes[0])
axes[0].set_title('Unweighted',fontdict= { 'fontsize': 16, 'fontweight':'bold'})
axes[0].set_xlabel("Degree")
axes[0].set_ylabel("Average degree connectivity")

sns.scatterplot(data=avg_weighted_degree_connectivity,
    ax=axes[1])
axes[1].set_title('Weighted',fontdict= { 'fontsize': 16, 'fontweight':'bold'})
axes[1].set_xlabel("Degree")
axes[1].set_ylabel("Average weighted degree connectivity")


fig_knearest.figure.set_size_inches((15,5))

# Distance

## Average shortest path length

In [ ]:
avg_short_path = nx.average_shortest_path_length(G)
cprint("Average shortest path length: ", 'blue', end=' ')
cprint(round(avg_short_path,2), 'green')

if(math.isclose(math.log(G.number_of_nodes(),10),avg_short_path,abs_tol=0.5)):
    cprint("Small-word", 'yellow')
else:
    cprint("NOT Small-word ", 'yellow')



## Shortest path length distribution

In [ ]:
shortest_path_lengths = dict(nx.shortest_path_length(G))
#counting how many paths have certain length
shortest_length_counts = Counter()
for i in range(num_nodes):
    if(i!=0):
        shortest_length_counts.update(Counter(shortest_path_lengths[i].values()))

In [ ]:
#shortest path length minimo e massimo
min_length, max_length = min(shortest_length_counts.keys()), max(shortest_length_counts.keys())

#asse x: shortest path length
shortest_length_plot_x = list(range(min_length, max_length + 1))

#asse y: num of paths
shortest_length_plot_y = [shortest_length_counts.get(x, 0) for x in shortest_length_plot_x]

plt.bar(shortest_length_plot_x, shortest_length_plot_y)

plt.xlabel("Shortest path length")
plt.ylabel("Number of paths")

Function that returns the probability that a path has a certain length

In [ ]:
def get_shortest_path_length_probability(shortest_length_counts, min_length, max_length):
  length_prob = {}
  num_path = sum(shortest_length_counts.values())

  for length in range(min_length, max_length + 1):
    if length in shortest_length_counts:
      num_path_length = shortest_length_counts[length]
      prob = num_path_length/num_path
      length_prob[length] = prob
    else:
      length_prob[length] = 0
  
  return length_prob

In [ ]:
length_probability = get_shortest_path_length_probability(shortest_length_counts, min_length, max_length)

shortest_length_plot_y_prob = [length_probability[x] for x in shortest_length_plot_x]

plt.plot(shortest_length_plot_x, shortest_length_plot_y_prob)
plt.xlabel("Shortest path length")
plt.ylabel("Shortest path length probability")

# Diameter

Diameter of the largest strongly connected component

In [ ]:
diameter = nx.diameter(G.subgraph(largest_strongly))

In [ ]:
cprint("Diameter: ", 'blue', end=' ')
cprint(diameter, 'green')

# Clustering coefficient

## Triangle

Number of triangles that include a node as a vertex.<br>
Graph is trasformed to undirected.<br>
When calculating the number of triangles for the whole graph, each triangle is counted 3 times, one for each node.<br>
Self loops are ignored.

In [ ]:
triangles = nx.triangles(G.to_undirected())
num_triangles = sum(triangles.values())
cprint("Triangle counts: ", 'blue', end=' ')
cprint(int(num_triangles/3), 'green')

In [ ]:
clustering_coefficient = nx.clustering(G)
weighted_clustering_coefficient = nx.clustering(G, weight='weight')

In [ ]:
#Creating figure area
fig_clustering, axes = plt.subplots(nrows=1, ncols=2)

#Clustering min max
min_clustering, max_clustering = min(clustering_coefficient.keys()), max(clustering_coefficient.keys())
clustering_plot_x = list(range(min_clustering, max_clustering + 1))
clustering_plot_y = [clustering_coefficient[key] for key in clustering_plot_x]

axes[0].bar(clustering_plot_x, clustering_plot_y)
axes[0].set_xlabel('Node label')
axes[0].set_ylabel('Clustering coefficient')
axes[0].set_title('Unweighted',fontdict= { 'fontsize': 16, 'fontweight':'bold'})

#Weighted clustering min and max
min_clustering, max_clustering = min(weighted_clustering_coefficient.keys()), max(weighted_clustering_coefficient.keys())
clustering_plot_x = list(range(min_clustering, max_clustering + 1))
clustering_plot_y = [weighted_clustering_coefficient[key] for key in clustering_plot_x]

axes[1].bar(clustering_plot_x, clustering_plot_y)
axes[1].set_xlabel('Node label')
axes[1].set_ylabel('Weighted Clustering coefficient')
axes[1].set_title('Weighted',fontdict= { 'fontsize': 16, 'fontweight':'bold'})

fig_clustering.figure.set_size_inches((15,5))

In [ ]:
avg_clustering = nx.average_clustering(G)
weighted_avg_clustering = nx.average_clustering(G, weight='weight')
cprint("Average weighted clustering coefficient:", 'blue', end=' ')
cprint(round(weighted_avg_clustering, 4), 'green')
cprint("Average clustering coefficient:", 'blue', end=' ')
cprint(round(avg_clustering, 4), 'green')

# Communities

## Girvan–Newman
Funzione che trova le comunità in un grafo G usando il metodo di Girvan-Newman.

L'algoritmo di Girvan-Newman rileva le comunità rimuovendo progressivamente gli archi dal grafo originale. L'algoritmo rimuove l'arco con la più alta centralità (betweenness) tra gli elementi, ad ogni step. Quando il grafico si scompone in pezzi, viene esposta la struttura della comunità strettamente unita e il risultato può essere rappresentato come un dendrogramma.

In [ ]:
# https://stackoverflow.com/questions/59821151/plot-the-dendrogram-of-communities-found-by-networkx-girvan-newman-algorithm

from itertools import chain, combinations
from scipy.cluster.hierarchy import dendrogram

# get Graph and Girvan-Newman communities list
communities = list(nx.community.girvan_newman(G))

# building initial dict of node_id to each possible subset:
node_id = 0
init_node2community_dict = {node_id: communities[0][0].union(communities[0][1])}
for comm in communities:
    for subset in list(comm):
        if subset not in init_node2community_dict.values():
            node_id += 1
            init_node2community_dict[node_id] = subset

# turning this dictionary to the desired format in @mdml's answer
node_id_to_children = {e: [] for e in init_node2community_dict.keys()}
for node_id1, node_id2 in combinations(init_node2community_dict.keys(), 2):
    for node_id_parent, group in init_node2community_dict.items():
        if len(init_node2community_dict[node_id1].intersection(init_node2community_dict[node_id2])) == 0 and group == init_node2community_dict[node_id1].union(init_node2community_dict[node_id2]):
            node_id_to_children[node_id_parent].append(node_id1)
            node_id_to_children[node_id_parent].append(node_id2)

# also recording node_labels dict for the correct label for dendrogram leaves
node_labels = dict()
for node_id, group in init_node2community_dict.items():
    if len(group) == 1:
        node_labels[node_id] = list(group)[0]
    else:
        node_labels[node_id] = ''

# also needing a subset to rank dict to later know within all k-length merges which came first
subset_rank_dict = dict()
rank = 0
for e in communities[::-1]:
    for p in list(e):
        if tuple(p) not in subset_rank_dict:
            subset_rank_dict[tuple(sorted(p))] = rank
            rank += 1
subset_rank_dict[tuple(sorted(chain.from_iterable(communities[-1])))] = rank

# my function to get a merge height so that it is unique (probably not that efficient)
def get_merge_height(sub):
    sub_tuple = tuple(sorted([node_labels[i] for i in sub]))
    n = len(sub_tuple)
    other_same_len_merges = {k: v for k, v in subset_rank_dict.items() if len(k) == n}
    min_rank, max_rank = min(other_same_len_merges.values()), max(other_same_len_merges.values())
    range = (max_rank-min_rank) if max_rank > min_rank else 1
    return float(len(sub)) + 0.8 * (subset_rank_dict[sub_tuple] - min_rank) / range

# finally using @mdml's magic, slightly modified:
G           = nx.DiGraph(node_id_to_children)
nodes       = G.nodes()
leaves      = set( n for n in nodes if G.out_degree(n) == 0 )
inner_nodes = [ n for n in nodes if G.out_degree(n) > 0 ]

# Compute the size of each subtree
subtree = dict( (n, [n]) for n in leaves )
for u in inner_nodes:
    children = set()
    node_list = list(node_id_to_children[u])
    while len(node_list) > 0:
        v = node_list.pop(0)
        children.add( v )
        node_list += node_id_to_children[v]
    subtree[u] = sorted(children & leaves)

inner_nodes.sort(key=lambda n: len(subtree[n])) # <-- order inner nodes ascending by subtree size, root is last

# Construct the linkage matrix
leaves = sorted(leaves)
index  = dict( (tuple([n]), i) for i, n in enumerate(leaves) )
Z = []
k = len(leaves)
for i, n in enumerate(inner_nodes):
    children = node_id_to_children[n]
    x = children[0]
    for y in children[1:]:
        z = tuple(sorted(subtree[x] + subtree[y]))
        i, j = index[tuple(sorted(subtree[x]))], index[tuple(sorted(subtree[y]))]
        Z.append([i, j, get_merge_height(subtree[n]), len(z)]) # <-- float is required by the dendrogram function
        index[z] = k
        subtree[z] = list(z)
        x = z
        k += 1

# dendrogram
plt.figure()
dendrogram(Z, labels=[node_labels[node_id] for node_id in leaves])
plt.savefig('dendrogram.png')

KeyboardInterrupt: 

## QUALITY OF PARTITIONS

In [19]:

communities = nx.algorithms.community.centrality.girvan_newman(G)
max_modularity = 0
for x in communities:
   modularity_of_x = nx.algorithms.community.quality.modularity(G,x, weight='weight')
   if modularity_of_x >= max_modularity:
      max_modularity = modularity_of_x
      best_partition = x
   print(modularity_of_x)
print ('Max Mod:', max_modularity)
print ('Best Partition:', best_partition)   


0.0
0.34840580646161823
0.3870592307808582
0.3873558391370008
0.38647373140698094
0.3857358851475516
0.40248129484031264
0.40272937843746687
0.4008514763992258
0.4001221940542778
0.3994706985543327
0.3985288151920015
0.40662594725728
0.4185044892579557
0.4172194849323201
0.4134997400888763
0.414221918556339
0.4132199160235797
0.4170065281076017
0.4163580753738333
0.4168906576085154
0.41542086658373334
0.4123003749080545
0.4113408729480223
0.40881724194250973
0.4077043992902306
0.40641978757958547
0.40546175792576783
0.4037809608820017
0.4033509002367272
0.4058123017662374
0.4115769695187463
0.4098240049320295
0.4087406574809154
0.4073786270018211
0.4074827926665016
0.4042267506653352
0.40396537950459444
0.402687503594881
0.3964116633938936
0.39599922134632
0.3946374730592501
0.3916790700285765
0.3879923784596496
0.3878404978043988
0.3871773710853586
0.38386360241136264
0.38105164477216447
0.38029165257342434
0.3771904235707194
0.3746151882323864
0.3672127270467952
0.36611063222988227
0

In [ ]:

node_groups = []
k = 2

for com in itertools.islice(communities, k):
  node_groups.append(list(com))


color_map = []


for node in G:
  if node in node_groups[0]:
    color_map.append('blue')
  else: 
    color_map.append('green')  

plt.figure(5, figsize=(50, 50))
nx.draw(G, node_color=color_map, with_labels=True)
plt.show()

In [ ]:
partition1 = node_groups[0]
partition2 = node_groups[1]

is_part1 = nx.community.is_partition(G, partition1)
is_part2 = nx.community.is_partition(G, partition2)

print(partition1)
print(partition2)

In [ ]:
!pip install python-louvain

In [ ]:

import community
import matplotlib.cm as cm

# compute the best partition
partition_louvain = community.best_partition(G.to_undirected())

# draw the graph
pos_louvain = nx.spring_layout(G)
# color the nodes according to their partition
cmap_louvain = cm.get_cmap('viridis', max(partition_louvain.values()) + 1)

plt.figure(6, figsize=(50, 50))
nx.draw_networkx_nodes(G, pos_louvain, partition_louvain.keys(), node_size=500, cmap=cmap_louvain, node_color=list(partition_louvain.values()))
nx.draw_networkx_edges(G, pos_louvain, alpha=0.5)
plt.show()